# Support Vector Machines 
### with Hyperparameter Tuning

### Hinweis:
Anders als in den Praktika und Vorlesungen verwenden wir hier nicht `Scikit-learn`, sondern die Bibliothek `cuML` (CUDA Machine Learning). Diese ermöglicht es SVMs auf der GPU zu trainieren, was den Trainingsprozess extrem beschleunigt. An der Herangehensweise und der Art, wie wir die Hyperparameter tunen, ändert sich dadurch nichts.

In [1]:
import cudf
import cupy as cp
import numpy as np
import time
from cuml.svm import SVC
from cuml.metrics import accuracy_score
from sklearn.model_selection import ParameterSampler, ParameterGrid
from cuml.model_selection import GridSearchCV
import joblib

DATAPATH = '../Data/'
MODELPATH = '../Data/Models/SVM/'
MODELPATHSKLEARN = '../Data/Models/SVM/SVM_sklearn/'

train_set = cudf.read_csv(DATAPATH + 'train_set.csv')
train_labels = cudf.read_csv(DATAPATH + 'train_labels.csv')
test_set = cudf.read_csv(DATAPATH + 'test_set.csv')
test_labels = cudf.read_csv(DATAPATH + 'test_labels.csv')


# WITHOUT Hyperparameter Tuning

In [18]:
clf = SVC()

clf.fit(train_set, train_labels)

test_pred = clf.predict(test_set)
accuracy_score = accuracy_score(test_labels, test_pred)

joblib.dump(clf, MODELPATH + 'SVM_no_hyper.pkl')

print(f'Accuracy: {accuracy_score}')

Accuracy: 0.7532467246055603


# With (Random) Hyperparameter Tuning

In [3]:
param_distributions = [
    {
        'kernel': ['linear'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['poly'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True), 
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'degree': [2, 3, 4, 5, 6], # Macht natrülich nur Sinn, wenn degree > 1 da sonst linear
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['rbf'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    },
    {
        'kernel': ['sigmoid'],
        'C': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'tol': np.logspace(np.log10(0.0000001), np.log10(0.1), 7, endpoint=True),
        'gamma': np.logspace(np.log10(0.001), np.log10(1000), 7, endpoint=True),
        'coef0': np.linspace(-1, 1, 10),
        'max_iter': [1000, 2000, 3000, 4000, 5000],
        'class_weight': ['balanced'],
    }
]

In [3]:
n_iter = 100

best_accuracy = 0
best_params = None

start_time = time.time()

all_sampled_params = []
for params in param_distributions:
    sampler = ParameterSampler(params, n_iter=n_iter, random_state=42)
    sampled_params = list(sampler)
    all_sampled_params.extend(sampled_params)

clf = SVC()

counter = 0

for params in all_sampled_params:
    try:
        clf.set_params(**params)
        clf.fit(train_set, train_labels)
        predictions = clf.predict(test_set)
        accuracy = accuracy_score(test_labels, predictions)
        counter += 1

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
    except ValueError as e:
        print(f"Skipping invalid parameter combination: {params}")
        print(f"Error: {e}")


end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels) 
test_predictions = best_model.predict(test_set)
test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_sampl.pkl')

['../Data/Models/SVM/SVM_para_sampl.pkl']

In [4]:
print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")
print(f"Time taken: {end_time - start_time:.2f} seconds")
print(f"Test accuracy with best model: {test_accuracy}")

Best parameters: {'tol': np.float64(0.1), 'max_iter': 4000, 'kernel': 'poly', 'gamma': np.float64(0.1), 'degree': 5, 'class_weight': 'balanced', 'C': np.float64(1.0)}
Best accuracy: 0.7662337422370911
Time taken: 39.86 seconds
Test accuracy with best model: 0.701298713684082


# Further Hyperparameter Tuning with Parameter Grid

In [7]:
param_grid = {
        'kernel': ['poly'],
        'C': [0.01, 0.1, 0.8, 0.9, 1, 1.1, 1.2, 2, 5, 10], 
        'tol': [0.01, 0.01, 0.1],
        'gamma': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5],
        'degree': [4, 5, 6],
        'max_iter': np.linspace(3900, 4100, 10),
        'class_weight': ['balanced'],
    },

best_accuracy = 0
best_params = None

start_time = time.time()

for params in ParameterGrid(param_grid):
    clf.set_params(**params)
    clf.fit(train_set, train_labels)
    predictions = clf.predict(test_set)
    accuracy = accuracy_score(test_labels, predictions)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params    

end_time = time.time()

best_model = SVC(**best_params)
best_model.fit(train_set, train_labels)
test_predictions = best_model.predict(test_set)

test_accuracy = accuracy_score(test_labels, test_predictions)

joblib.dump(best_model, MODELPATH + 'SVM_para_grid.pkl')

['../Data/Models/SVM/SVM_para_grid.pkl']

In [8]:
print("\n" + "="*50)
print("GRID SEARCH RESULTS")
print("="*50)
print("\nBest Parameters Found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

print("\nModel Performance:")
print(f"  Validation Accuracy: {best_accuracy*100:.2f}%")

print(f"  Test Accuracy: {test_accuracy*100:.2f}%")
print(f"\nTraining Time: {end_time - start_time:.2f} seconds")
print("="*50)


GRID SEARCH RESULTS

Best Parameters Found:
  C: 1.2
  class_weight: balanced
  degree: 4
  gamma: 0.07
  kernel: poly
  max_iter: 3900.0
  tol: 0.1

Model Performance:
  Validation Accuracy: 78.57%
  Test Accuracy: 66.88%

Training Time: 442.50 seconds


# (Random) Hyperparameter Tuning with Cross-Validation

In [4]:
class CuMLRandomizedSearchCV:

    model = SVC()

    def __init__(self, param_distributions, n_iter=10, cv_method='kfold', k=5, random_state=42):
        self.param_distributions = param_distributions
        self.n_iter = n_iter
        self.cv_method = cv_method
        self.k = k
        self.random_state = random_state
        self.results = []
        self.best_params = None
        self.best_score = None

    def _sample_parameters(self):
        """Sample parameter combinations from the given distributions."""
        for param_distribution in self.param_distributions:
            sampler = ParameterSampler(param_distribution, n_iter=self.n_iter, random_state=self.random_state)
            for sampled_params in sampler:
                yield sampled_params

    def _train_and_evaluate(self, X, y, sampled_params):
        """Train and evaluate a model with given parameters using the chosen CV method."""
        scores = []
        
        if self.cv_method == 'kfold':
            scores = self._kfold_cv(X, y, sampled_params)
        elif self.cv_method == 'stratified':
            scores = self._stratified_kfold_cv(X, y, sampled_params)
        elif self.cv_method == 'loocv':
            scores = self._loocv(X, y, sampled_params)
        else:
            raise ValueError("Invalid cv_method. Choose 'kfold', 'stratified', or 'loocv'.")
        
        return np.mean(scores)

    def _kfold_cv(self, X, y, sampled_params):
        """Perform k-fold cross-validation."""
        from sklearn.model_selection import KFold

        kf = KFold(n_splits=self.k, shuffle=True, random_state=self.random_state)
        scores = []
        for fold, (train_index, val_index) in enumerate(kf.split(X)):
            print(f"  Training and evaluating fold {fold+1}/{self.k}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            CuMLRandomizedSearchCV.model.set_params(**sampled_params)
            CuMLRandomizedSearchCV.model.fit(X_train, y_train)
            score = CuMLRandomizedSearchCV.model.score(X_val, y_val)
            scores.append(score)
        return scores

    def _stratified_kfold_cv(self, X, y, sampled_params):
        """Perform stratified k-fold cross-validation."""
        from sklearn.model_selection import StratifiedKFold
        from sklearn.svm import SVC
        X = X.to_pandas()
        y = y.to_pandas()
        y.to_numpy()
        #y_flat = cp.array([], dtype=np.float32)
        #for i in y.to_cupy():
        #        y_flat = cp.append(y, i[0])
        model = SVC()
        skf = StratifiedKFold(n_splits=self.k, random_state=self.random_state, shuffle=True)
        scores = []
        for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
            print(f"  Training and evaluating fold {fold+1}/{self.k}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            model.set_params(**sampled_params)
            model.fit(X_train, y_train)
            score = model.score(X_val, y_val)
            scores.append(score)
        return scores

    def _loocv(self, X, y, sampled_params):
        """Perform Leave-One-Out cross-validation."""
        from sklearn.model_selection import LeaveOneOut
        
        loo = LeaveOneOut()
        scores = []
        for i, (train_index, val_index) in enumerate(loo.split(X)):
            print(f"  Training and evaluating sample {i+1}/{len(X)}...")
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            CuMLRandomizedSearchCV.model.set_params(**sampled_params)
            CuMLRandomizedSearchCV.model.fit(X_train, y_train)
            score = CuMLRandomizedSearchCV.model.score(X_val, y_val)
            scores.append(score)
        return scores

    def fit(self, X, y):
        """Fit the randomized search CV to the data."""
        print("Starting Randomized Search CV...")
        for i, sampled_params in enumerate(self._sample_parameters()):
            print(f"Evaluating parameter set {i+1}/{self.n_iter * len(self.param_distributions)}: {sampled_params}")
            avg_score = self._train_and_evaluate(X, y, sampled_params)
            print(f"  Average CV score: {avg_score:.4f}")
            self.results.append({'params': sampled_params, 'score': avg_score})

        best_result = max(self.results, key=lambda x: x['score'])
        self.best_params = best_result['params']
        self.best_score = best_result['score']
        self.X_train = X
        self.y_train = y
        print("Randomized Search CV complete.")
        print(f"Best parameters found: {self.best_params}")
        print(f"Best CV score: {self.best_score:.4f}")
        return self

    def predict(self, train_set, train_labels, test_set):
        """Predict using the best found parameters."""
        print("Training final model with best parameters...")
        final_model = SVC(**self.best_params)
        final_model.fit(train_set, train_labels)
        print("Final model training complete.")
        return final_model.predict(test_set)

    def score(self, X, y):
        """Score the model's performance."""
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)
    
    def get_best_params(self):
        """Return the best parameters found by the randomized search."""
        return self.best_params
    
    def get_best_score(self):
        """Return the best score found by the randomized search."""
        return self.best_score
    
    def get_results(self):
        """Return the results of the randomized search."""
        return self.results
    
    def get_best_model(self):
        """Return the best model found by the randomized search."""
        return SVC(**self.best_params)
    
    def save_best_model(self, train_set, train_labels):
        """Save the best model to a file."""
        final_model = SVC(**self.best_params)
        final_model.fit(train_set, train_labels)
        joblib.dump(final_model, DATAPATH + 'SVM_rnd_search_CV.pkl')

In [4]:
train_set_a = train_set.to_pandas()
train_labels_a = train_labels.to_pandas()

train_labels_a.to_numpy()

array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
    

In [5]:
from sklearn.model_selection import RandomizedSearchCV as RandomizedSearchCV_sklearn
from sklearn.svm import SVC as SVC_sklearn
from sklearn.model_selection import StratifiedKFold


skf = StratifiedKFold(n_splits=5)
svc_start = SVC_sklearn()
rnd_search = RandomizedSearchCV_sklearn(svc_start, param_distributions, cv=skf, n_iter=10)
rnd_search.fit(train_set, train_labels)
print()

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().

In [5]:
searcher = CuMLRandomizedSearchCV(param_distributions=param_distributions, n_iter=20, cv_method='stratified', k=5, random_state=60)
searcher.fit(train_set, train_labels)

Starting Randomized Search CV...
Evaluating parameter set 1/80: {'tol': np.float64(1e-07), 'max_iter': 5000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(1.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7437
Evaluating parameter set 2/80: {'tol': np.float64(1e-06), 'max_iter': 4000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(100.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.5413
Evaluating parameter set 3/80: {'tol': np.float64(1e-07), 'max_iter': 4000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(0.01)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and e

/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/svm/_base.py:304: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d ar

  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7150
Evaluating parameter set 6/80: {'tol': np.float64(0.1), 'max_iter': 2000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(0.001)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7150
Evaluating parameter set 7/80: {'tol': np.float64(9.999999999999999e-06), 'max_iter': 2000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(1000.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.5087
Evaluating parameter set 8/80: {'tol': np.float64(9.999999999999999e-06), 'max_iter': 2000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.floa

/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWar

  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7425
Evaluating parameter set 12/80: {'tol': np.float64(1e-06), 'max_iter': 3000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(0.001)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7150
Evaluating parameter set 13/80: {'tol': np.float64(0.1), 'max_iter': 4000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(0.1)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7450
Evaluating parameter set 14/80: {'tol': np.float64(0.01), 'max_iter': 3000, 'kernel': 'linear', 'class_weight': 'balanced', 'C': np.float64(1.0)}
  Training and evaluati

...

/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWar

  Average CV score: 0.6713
Evaluating parameter set 73/80: {'tol': np.float64(0.01), 'max_iter': 4000, 'kernel': 'sigmoid', 'gamma': np.float64(0.1), 'coef0': np.float64(-0.33333333333333337), 'class_weight': 'balanced', 'C': np.float64(1.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7225
Evaluating parameter set 74/80: {'tol': np.float64(0.0001), 'max_iter': 4000, 'kernel': 'sigmoid', 'gamma': np.float64(0.1), 'coef0': np.float64(0.7777777777777777), 'class_weight': 'balanced', 'C': np.float64(1.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.6400
Evaluating parameter set 75/80: {'tol': np.float64(0.01), 'max_iter': 1000, 'kernel': 'sigmoid', 'gamma': n

/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWar

  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.7150
Evaluating parameter set 78/80: {'tol': np.float64(0.001), 'max_iter': 5000, 'kernel': 'sigmoid', 'gamma': np.float64(10.0), 'coef0': np.float64(0.33333333333333326), 'class_weight': 'balanced', 'C': np.float64(1000.0)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and evaluating fold 5/5...
  Average CV score: 0.5825
Evaluating parameter set 79/80: {'tol': np.float64(1e-07), 'max_iter': 3000, 'kernel': 'sigmoid', 'gamma': np.float64(1.0), 'coef0': np.float64(-0.33333333333333337), 'class_weight': 'balanced', 'C': np.float64(0.1)}
  Training and evaluating fold 1/5...
  Training and evaluating fold 2/5...
  Training and evaluating fold 3/5...
  Training and evaluating fold 4/5...
  Training and 

/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/miniconda3/envs/rapids-24.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#svc_rnd_cv = searcher.get_best_model()
#svc_rnd_cv.fit(train_set, train_labels)


test_model = SVC(**para)
test_model.fit(train_set, train_labels) 
#test_predictions = test_model.predict(test_set)
#test_accuracy = accuracy_score(test_labels, test_predictions)

[D] [16:17:59.862905] /opt/conda/conda-bld/work/cpp/src/svm/workingset.h:100 Creating working set with 800 elements
[D] [16:18:11.692424] /opt/conda/conda-bld/work/cpp/src/svm/smosolver.h:281 Solver is not converging monotonically. This might be caused by insufficient normalization of the feature columns. In that case MinMaxScaler((0,1)) could help. Alternatively, for nonlinear kernels, you can try to increase the gamma parameter. To limit execution time, you can also adjust the number of iterations using the max_iter parameter.
[D] [16:18:53.515093] /opt/conda/conda-bld/work/cpp/src/svm/smosolver.cuh:202 SMO iteration 500, diff 10.817611
[D] [16:19:45.745670] /opt/conda/conda-bld/work/cpp/src/svm/smosolver.cuh:202 SMO iteration 1000, diff 1.305868


In [38]:
print(f"Test accuracy with best model: {test_accuracy}")
print(test_model.)

Test accuracy with best model: 0.5


In [34]:
svc_rnd_cv_predictions = svc_rnd_cv.predict(test_set)
svc_rnd_cv_accuracy = accuracy_score(test_labels, svc_rnd_cv_predictions)

print(f"Test accuracy with best model: {svc_rnd_cv_accuracy}")

Test accuracy with best model: 0.5


# Further Tuning with GridSearchCV

In [ ]:
param_gridCV = {
        'kernel': ['poly'],
        'C': [0.01, 0.1, 0.8, 0.9, 1, 1.1, 1.2, 2, 5, 10], 
        'tol': [0.01, 0.01, 0.1],
        'gamma': [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5],
        'degree': [4, 5, 6],
        'max_iter': np.linspace(3900, 4100, 10),
        'class_weight': ['balanced'],
    }

In [ ]:
svm_grid_search = GridSearchCV(SVC(), param_gridCV, cv=5)
svm_grid_search.fit(train_set, train_labels)

print("\n" + "="*50)
print("GRID SEARCH RESULTS")
print("="*50)
print("\nBest Parameters Found:")
for param, value in svm_grid_search.best_params_.items():
    print(f"  {param}: {value}")

print("\nModel Performance:")
print(f"  Validation Accuracy: {svm_grid_search.best_score_*100:.2f}%")
print("="*50)

best_model = svm_grid_search.best_estimator_
test_predictions = best_model.predict(test_set)
test_accuracy = accuracy_score(test_labels, test_predictions)

print(f"Test Accuracy: {test_accuracy*100:.2f}%")

joblib.dump(best_model, MODELPATH + 'SVM_para_grid_CV.pkl')